In [2]:
from scipy.io import loadmat
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [133]:
def load_data(filename):
    """
    Load data from a .mat file.
    """
    data = loadmat(filename)
    return data['lsl_data'][:,:-1], data['marker_data'].reshape(-1,4)

In [155]:
def filter_data(raw_data, marker_data):
    cleaned_data = []
    labels = []
    for start, label, end, isbad in marker_data:
        if abs(isbad-99) < 1e-3:
            continue
        cleaned_data.append([])
        mask = (raw_data[:, 0] >= start) & (raw_data[:, 0] <= end)
        cleaned_data[-1].extend(raw_data[mask][:1380])
        labels.append(label)
    return np.array(cleaned_data), labels

In [153]:
raw_data, marker_data = load_data('raw_data/lsl_data_2023-10-27-16-22-14.mat')
filter_data, labels = filter_data(raw_data, marker_data)

In [158]:
all_data = []
all_labels = []
for filename in os.listdir('raw_data'):
    if filename.endswith('.mat'):
        raw_data, marker_data = load_data('raw_data/' + filename)
        cleaned_data, labels = filter_data(raw_data, marker_data)
        all_data.append(cleaned_data)
        all_labels += labels
all_labels = np.array(all_labels)
all_data = np.concatenate(all_data)

In [163]:
np.save( 'processed_data/all_data.npy', all_data)
np.save('processed_data/all_labels.npy', all_labels)